In [1]:
#Bibliothéques
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df_scores = pd.read_excel(r"C:\Users\LOQ\Downloads\projet_power_pi\scores_2019_cleaned_dataset.xlsx")
df_athletes = pd.concat([
    pd.read_excel(r"C:\Users\LOQ\Downloads\projet_power_pi\athletes_women_2019_cleaned (1).xlsx"),
    pd.read_excel(r"C:\Users\LOQ\Downloads\projet_power_pi\athletes_men_2019_cleaned (1).xlsx")
])

In [27]:
df_scores.head()

,ordinal,lane,rank,heat,points,time,workoutrank,competitorid,number of records,time_td,time_seconds
0,1,26,1.0,3,100,0 days 00:15:07.700000,1,153604,1,0.010506,907.70
1,2,20,1.0,2,100,0 days 00:05:37.020000,1,153604,1,0.003901,337.02
2,3,25,17.0,1,68,0 days 00:25:50.090000,17,153604,1,0.017941,1550.09
3,4,10,21.0,2,40,0 days 00:01:50.080000,21,153604,1,0.001274,110.08
4,5,8,2.0,2,96,0 days 00:08:29.770000,2,153604,1,0.005900,509.77


In [29]:
df_athletes.head()

,height,affiliateid,countryoforiginname,weight,affiliatename,status,bibid,profilepics3key,competitorid,firstname,gender,age,lastname,countryoforigincode,competitorname,overallrank,overallscore,division,number of records
0,1.65,44632,Australia,58.1,CrossFit Mayhem,ACTIVE,17,b8a69-P163097_3-184.jpg,163097,Tia-Clair,FEMALE,25,Toomey,AU,Tia-Clair Toomey,1,1071,Women,1
1,1.65,44731,Norway,59.0,CrossFit Oslo,ACTIVE,122,df164-P120480_7-184.jpg,120480,Kristin,FEMALE,33,Holte,NO,Kristin Holte,2,876,Women,1
2,1.65,44663,New Zealand,61.2,CrossFit Yas,ACTIVE,123,101af-P264512_2-184.jpg,264512,Jamie,FEMALE,28,Greene,NZ,Jamie Greene,3,851,Women,1
3,1.74,44608,Iceland,68.0,CrossFit New England,ACTIVE,15,ee582-P55121_22-184.jpg,55121,KatrÃ­n Tanja,FEMALE,26,Davidsdottir,IS,Katrin Tanja Davidsdottir,4,784,Women,1
4,1.62,44589,United States,63.0,CrossFit Spot,ACTIVE,103,17c8c-P305891_3-184.jpg,305891,Karissa,FEMALE,30,Pearce,US,Karissa Pearce,5,714,Women,1


In [3]:
# 2️⃣ Merge datasets
# -----------------------------
df = df_scores.merge(df_athletes, on='competitorid', how='left')

In [4]:
df.head()

,ordinal,lane,rank,heat,points,time,workoutrank,competitorid,number of records_x,time_td,...,firstname,gender,age,lastname,countryoforigincode,competitorname,overallrank,overallscore,division,number of records_y
0,1,26,1.0,3,100,0 days 00:15:07.700000,1,153604,1,0.010506,...,Mathew,MALE,29,Fraser,US,Mathew Fraser,1,984,Men,1
1,2,20,1.0,2,100,0 days 00:05:37.020000,1,153604,1,0.003901,...,Mathew,MALE,29,Fraser,US,Mathew Fraser,1,984,Men,1
2,3,25,17.0,1,68,0 days 00:25:50.090000,17,153604,1,0.017941,...,Mathew,MALE,29,Fraser,US,Mathew Fraser,1,984,Men,1
3,4,10,21.0,2,40,0 days 00:01:50.080000,21,153604,1,0.001274,...,Mathew,MALE,29,Fraser,US,Mathew Fraser,1,984,Men,1
4,5,8,2.0,2,96,0 days 00:08:29.770000,2,153604,1,0.005900,...,Mathew,MALE,29,Fraser,US,Mathew Fraser,1,984,Men,1


In [5]:
df.describe()

,ordinal,lane,rank,heat,points,workoutrank,competitorid,number of records_x,time_td,time_seconds,height,affiliateid,weight,age,overallrank,overallscore,number of records_y
count,4213.000000,4213.000000,4213.000000,4213.000000,4213.000000,4213.000000,4.213000e+03,4213.0,4213.000000,4213.000000,4213.000000,4213.000000,4213.000000,4213.000000,4213.000000,4213.000000,4213.0
mean,6.034180,6.386423,10.719915,0.508901,19.004747,11.566105,4.241658e+05,1.0,0.006722,580.753131,1.655106,40176.543081,74.596250,32.743888,55.547591,176.184192,1.0
std,3.345032,10.980308,15.880164,0.700515,32.190398,15.482681,3.765305e+05,0.0,0.004063,351.016089,0.263568,13547.308321,12.120912,9.722023,44.121674,242.987260,0.0
min,1.000000,0.000000,1.000000,0.000000,-63.000000,1.000000,1.690000e+03,1.0,0.001010,87.300000,0.030000,0.000000,50.800000,17.000000,1.000000,-63.000000,1.0
25%,3.000000,0.000000,5.000000,0.000000,0.000000,8.000000,1.202470e+05,1.0,0.005900,509.770000,1.580000,44630.000000,64.000000,27.000000,10.000000,-17.000000,1.0
50%,6.000000,0.000000,6.000000,0.000000,0.000000,8.000000,2.977530e+05,1.0,0.005900,509.770000,1.710000,44709.000000,73.000000,30.000000,50.000000,53.000000,1.0
75%,9.000000,10.000000,8.000000,1.000000,37.000000,8.000000,6.639790e+05,1.0,0.005900,509.770000,1.770000,44804.000000,86.200000,35.000000,94.000000,329.000000,1.0
max,12.000000,68.000000,133.000000,3.000000,100.000000,133.000000,1.697328e+06,1.0,0.031250,2700.000000,1.950000,44977.000000,102.100000,63.000000,144.000000,1071.000000,1.0


In [6]:
# 3️⃣ Compute past performance metrics
# -----------------------------
past_perf = df.groupby('competitorid').agg(
    avg_rank=('rank', 'mean'),
    avg_points=('points', 'mean'),
    top10_finishes=('rank', lambda x: (x <= 10).sum())
).reset_index()

df = df.merge(past_perf, on='competitorid', how='left')

In [31]:
df.head()

,ordinal,lane,rank,heat,points,time,workoutrank,competitorid,number of records_x,time_td,...,lastname,countryoforigincode,competitorname,overallrank,overallscore,division,number of records_y,avg_rank,avg_points,top10_finishes
0,1,26,1.0,3,100,0 days 00:15:07.700000,1,153604,1,0.010506,...,Fraser,US,Mathew Fraser,1,984,Men,1,5.666667,82.0,9
1,2,20,1.0,2,100,0 days 00:05:37.020000,1,153604,1,0.003901,...,Fraser,US,Mathew Fraser,1,984,Men,1,5.666667,82.0,9
2,3,25,17.0,1,68,0 days 00:25:50.090000,17,153604,1,0.017941,...,Fraser,US,Mathew Fraser,1,984,Men,1,5.666667,82.0,9
3,4,10,21.0,2,40,0 days 00:01:50.080000,21,153604,1,0.001274,...,Fraser,US,Mathew Fraser,1,984,Men,1,5.666667,82.0,9
4,5,8,2.0,2,96,0 days 00:08:29.770000,2,153604,1,0.005900,...,Fraser,US,Mathew Fraser,1,984,Men,1,5.666667,82.0,9


In [7]:
# 4️⃣ Select features & target
# -----------------------------
features = ['age', 'weight', 'height', 'gender', 'division', 'avg_points']
target = 'top10_finishes'

X = df[features]
y = df[target]

In [9]:
# 5️⃣ Preprocessing
# -----------------------------
numeric_features = ['age','weight','height','avg_points']
categorical_features = ['gender','division']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)


In [10]:
# 6️⃣ Train-test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -----------------------------
# 7️⃣ Random Forest Pipeline
# -----------------------------
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(
        n_estimators=200, 
        random_state=42, 
        max_depth=10
    ))
])

# -----------------------------
# 8️⃣ Train model
# -----------------------------
model.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [11]:
# 9️⃣ Evaluate model
# -----------------------------
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")

# -----------------------------
# 10️⃣ Predict new athletes
# -----------------------------
new_athletes = pd.DataFrame([
    {'age': 28, 'weight': 70, 'height': 1.75, 'gender': 'MALE', 'division': 'Individual', 'avg_points': 220},
    {'age': 32, 'weight': 60, 'height': 1.65, 'gender': 'FEMALE', 'division': 'Individual', 'avg_points': 210},
    {'age': 40, 'weight': 80, 'height': 1.80, 'gender': 'MALE', 'division': 'Team', 'avg_points': 180},
    {'age': 25, 'weight': 55, 'height': 1.60, 'gender': 'FEMALE', 'division': 'Team', 'avg_points': 190}
])

predictions = model.predict(new_athletes)

# Convert predictions to integers
predictions_int = [int(round(p)) for p in predictions]

for athlete, pred in zip(new_athletes.to_dict(orient='records'), predictions_int):
    print(f"Athlete: {athlete} --> Predicted top 10 finishes: {pred}")

Mean Squared Error (MSE): 0.014720824401244947
R² Score: 0.99318324992251
Athlete: {'age': 28, 'weight': 70, 'height': 1.75, 'gender': 'MALE', 'division': 'Individual', 'avg_points': 220} --> Predicted top 10 finishes: 9
Athlete: {'age': 32, 'weight': 60, 'height': 1.65, 'gender': 'FEMALE', 'division': 'Individual', 'avg_points': 210} --> Predicted top 10 finishes: 11
Athlete: {'age': 40, 'weight': 80, 'height': 1.8, 'gender': 'MALE', 'division': 'Team', 'avg_points': 180} --> Predicted top 10 finishes: 8
Athlete: {'age': 25, 'weight': 55, 'height': 1.6, 'gender': 'FEMALE', 'division': 'Team', 'avg_points': 190} --> Predicted top 10 finishes: 10


In [12]:
import joblib

# After training your pipeline model
joblib.dump(model, "athlete_model.pkl")
print("Model saved!")


Model saved!


In [13]:
pip install flask

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\LOQ\Desktop\Summer Internship\venv310\Scripts\python.exe -m pip install --upgrade pip
